In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense

vocab_size = 25000  # 사용할 단어 개수
maxlen = 200  # Bi-LSTM에 들어갈 최대 input sequence의 수 (단어 수)

In [2]:
# input layer -> Bi-LSTM -> Dense(output)
inputs = Input(shape=(None,), dtype="int32")

x = Embedding(vocab_size, 100)(inputs) ## text -> vector (100d)
x = Bidirectional(LSTM(128))(x) # # of hidden layer node = 128 (hyper-parameter)
x = Bidirectional(LSTM(128))(x) ## Multi-Layer Bi-LSTM

outputs = Dense(1, activation="sigmoid")(x) # [0, 1] ## binary classification
model = Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2500000   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 2,734,753
Trainable params: 2,734,753
Non-trainable params: 0
_________________________________________________________________


In [3]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=vocab_size)

print(len(X_train), "Training sequences")
print(len(X_test), "Test sequences")

X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen) # zero-padding
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

17473536/17464789 [==============================] - 0s 0us/step
25000 Training sequences
25000 Test sequences


In [4]:
# training setup
from tensorflow.keras.optimizers import SGD

## config variables
batch_size = 128
lr = 0.1
epochs = 15

# Set optimizer, loss function, metrics
optimizer = SGD(lr, momentum=0.9)
loss_fn = "binary_crossentropy" # binary-classification
metrics = ["accuracy"]

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=metrics)

In [5]:
model.fit(X_train, y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
196/196 [==============================] - 33s 126ms/step - loss: 0.6929 - accuracy: 0.5125 - val_loss: 0.6886 - val_accuracy: 0.5254
Epoch 2/5
196/196 [==============================] - 23s 119ms/step - loss: 0.6743 - accuracy: 0.5774 - val_loss: 0.6709 - val_accuracy: 0.6196
Epoch 3/5
196/196 [==============================] - 23s 120ms/step - loss: 0.6413 - accuracy: 0.6284 - val_loss: 0.5214 - val_accuracy: 0.7376
Epoch 4/5
196/196 [==============================] - 23s 120ms/step - loss: 0.6268 - accuracy: 0.6465 - val_loss: 0.5929 - val_accuracy: 0.6733
Epoch 5/5
196/196 [==============================] - 23s 119ms/step - loss: 0.6251 - accuracy: 0.6611 - val_loss: 0.5418 - val_accuracy: 0.7236


In [6]:
model.evaluate(X_test, y_test)

782/782 [==============================] - 26s 33ms/step - loss: 0.5418 - accuracy: 0.7236


[0.5418499708175659, 0.7235599756240845]